In [1]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
from tqdm import tqdm
from itertools import islice
import pandas as pd
from collections import defaultdict
import math
from cleantext import clean
import emoji
import re
import ast
import os


Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
print(os.getpid())

1360


In [3]:
real_small = pd.read_csv('/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/real.csv')
real_big = pd.read_csv('/mntssd/mnt3/shanshanbai/my_storage_from_qian/Data_Train_Test/first_paper/single_imbalance_noduplicate_test.csv')

/tmp/ipykernel_1360/1122205988.py:2: DtypeWarning: Columns (12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  real_big = pd.read_csv('/mntssd/mnt3/shanshanbai/my_storage_from_qian/Data_Train_Test/first_paper/single_imbalance_noduplicate_test.csv')


In [4]:
# Remove rows from df2 if building_id is in df1
real_big_filtered = real_big[~real_big['building_id'].isin(real_small['building_id'])]
# Filter rows where language is 'en'
real_big_filtered = real_big_filtered[real_big_filtered['language'] == 'en']
train = real_big_filtered.original_tweet.to_list()

In [5]:
def preprocess_dataset(raw_sentences):
    tokenized_sentences = []
    for sentence in raw_sentences:
        sentence = sentence.lower()
        sentence = re.sub(r"http\S+|www\S+|https\S+", "", sentence, flags=re.MULTILINE)
        sentence = re.sub(r"@\w+|#\w+", "", sentence)
        sentence = re.sub(r"[^a-zA-Z\s]", "", sentence)
        tokens = sentence.split()
        tokens = ["<s>"] + tokens + ["</s>"]
        tokenized_sentences.append(tokens)
    return tokenized_sentences

# Preprocess dataset
tokenized_sentences = preprocess_dataset(train)

In [6]:
def train_unigram_model(tokenized_sentences):
    """
    Train a unigram language model by counting token frequencies.
    Args:
        tokenized_sentences (list of list of str): List of tokenized sentences.
    Returns:
        unigram_counts (dict): Count of each unigram (word).
        total_tokens (int): Total number of tokens in the dataset.
    """
    unigram_counts = defaultdict(int)
    total_tokens = 0

    for sentence in tokenized_sentences:
        for token in sentence:
            unigram_counts[token] += 1
            total_tokens += 1

    return unigram_counts, total_tokens

# Train unigram model
unigram_counts, total_tokens = train_unigram_model(tokenized_sentences)


In [7]:
def calculate_unigram_probabilities(unigram_counts, total_tokens):
    unigram_probabilities = {}
    for token, count in unigram_counts.items():
        unigram_probabilities[token] = count / total_tokens
    return unigram_probabilities

# Calculate unigram probabilities
unigram_probabilities = calculate_unigram_probabilities(unigram_counts, total_tokens)


In [8]:
# Save n-gram probabilities
with open("/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/trained_models/unigram_model.json", "w") as f:
    json.dump(unigram_probabilities, f)

# # Load n-gram probabilities
# with open("ngram_model.json", "r") as f:
#     loaded_ngram_probs = json.load(f)


In [18]:
def calculate_perplexity(dataset, unigram_probs):
    total_log_prob = 0
    word_count = 0

    for sentence in dataset:
        tokens = sentence.split()
        for token in tokens:
            prob = unigram_probs.get(token, 1e-8)  # Handle unseen words with small probability
            total_log_prob += math.log10(prob)  # Log probability (base 2)
            word_count += 1

    if word_count == 0:
        return float("inf"), float("inf")  # Avoid division by zero

    perplexity = 10 ** (-total_log_prob / word_count)
    log_perplexity = math.log10(perplexity)  # Logarithmized perplexity (base 2)

    return perplexity, log_perplexity


In [19]:
real = pd.read_csv('/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/real.csv')
synthetic = pd.read_csv('/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/synthetic.csv')
real['tweets'] = real['tweet_no_url'].apply(ast.literal_eval)
synthetic['tweets'] = synthetic['tweets'].apply(ast.literal_eval)
# real['tweets'] = real['tweet_no_url'].apply(ast.literal_eval)
# synthetic['tweets'] = synthetic['tweets'].apply(ast.literal_eval)
# real = real[real['language'] == 'en']
# synthetic = synthetic[synthetic['language'] == 'en']


real_data = [tweet for sublist in real['tweets'] for tweet in sublist]
synthetic_data = [tweet for sublist in synthetic['tweets'] for tweet in sublist]

def clean_dataset(raw_sentences):
    tokenized_sentences = []
    for sentence in raw_sentences:
        sentence = sentence.lower()
        sentence = re.sub(r"http\S+|www\S+|https\S+", "", sentence, flags=re.MULTILINE)
        sentence = re.sub(r"@\w+|#\w+", "", sentence)
        sentence = re.sub(r"[^a-zA-Z\s]", "", sentence)
        tokenized_sentences.append(sentence)
    return tokenized_sentences

# Preprocess dataset
real_data = clean_dataset(real_data)
synthetic_data = clean_dataset(synthetic_data)

# Calculate perplexity
smoothed_probs = calculate_unigram_probabilities(unigram_counts, total_tokens)
synthetic_perplexity = calculate_perplexity(synthetic_data, smoothed_probs)
real_perplexity = calculate_perplexity(real_data, smoothed_probs)

print(f"Synthetic Data Perplexity: {synthetic_perplexity}")
print(f"Real Data Perplexity: {real_perplexity}")

Synthetic Data Perplexity: (31169.41253464588, 4.493728616990799)
Real Data Perplexity: (22169.78239835986, 4.345761430432695)
